In [1]:
from __future__ import division
# 多任务学习，交替训练，联合训练

import os
import tensorflow as tf
from PIL import Image
from nets import nets_factory
import numpy as np


RuntimeError: module compiled against API version 0xc but this version of numpy is 0xa

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xa

In [4]:
# 不同字符数量
CHAR_SET_LEN = 10
# 图片高度
IMAGE_HEIGHT = 60
# 图片高度
IMAGE_WIDTH = 160
# 批次
BATCH_SIZE = 25
# tfrecords文件存放路径
TFRECORD_FILE = ['captcha/images_train_00000-of-00002.tfrecord', 'captcha/images_train_00001-of-00002.tfrecord']

# placeholder
x = tf.placeholder(tf.float32, [None, 224, 224])
y0 = tf.placeholder(tf.float32, [None])
y1 = tf.placeholder(tf.float32, [None])
y2 = tf.placeholder(tf.float32, [None])
y3 = tf.placeholder(tf.float32, [None])


# 学习率
lr = tf.Variable(0.003, dtype=tf.float32)

# 从tfrecord读出数据
def read_and_decode(filename):
    # 根据文件名生成一个队列
    filename_queue = tf.train.string_input_producer(filename)
    reader = tf.TFRecordReader()
    # 返回文件名和文件
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example,
                                      features={
                                          'image': tf.FixedLenFeature([], tf.string),
                                          'label0': tf.FixedLenFeature([], tf.int64),
                                          'label1': tf.FixedLenFeature([], tf.int64),
                                          'label2': tf.FixedLenFeature([], tf.int64),
                                          'label3': tf.FixedLenFeature([], tf.int64),
                                      })
    
    # 获取图片数据
    image = tf.decode_raw(features['image'], tf.uint8)
    
    # tf.train.shuffle_batch 必须确定shape
    image = tf.reshape(image, [224, 224])
    # 图片预处理
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.subtract(image, 0.5)
    image = tf.multiply(image, 2.0)
    
    # 获取label
    label0 = tf.cast(features['label0'], tf.int32)
    label1 = tf.cast(features['label1'], tf.int32)
    label2 = tf.cast(features['label2'], tf.int32)
    label3 = tf.cast(features['label3'], tf.int32)
    
    return image, label0, label1, label2, label3



In [ ]:
# 获取图片数据和标签
image, label0, label1, label2, label3 = read_and_decode(TFRECORD_FILE)

# 使用shuffle_batch可以随机打乱
image_batch, label_batch0, label_batch1, label_batch2, label_batch3 = tf.train.shuffle_batch([image, label0, label1, label2, label3], batch_size = BATCH_SIZE, capacity = 50000, min_after_dequeue=10000, num_threads=2)
# print image_batch.shape, label_batch0.shape

train_network_fn = nets_factory.get_network_fn('alexnet_v2', num_classes=CHAR_SET_LEN, weight_decay=0.0005, is_training=True)

with tf.Session() as sess:
    # inputs: a tensor of size [batch_size, height, width, channels]
    X = tf.reshape(x, [BATCH_SIZE, 224, 224, 1])
    logits0, logits1, logits2, logits3, end_points = train_network_fn(X)
    
    # 把标签转换成one_hot格式
    one_hot_label0 = tf.one_hot(indices=tf.cast(y0, tf.int32), depth=CHAR_SET_LEN)
    one_hot_label1 = tf.one_hot(indices=tf.cast(y1, tf.int32), depth=CHAR_SET_LEN)
    one_hot_label2 = tf.one_hot(indices=tf.cast(y2, tf.int32), depth=CHAR_SET_LEN)
    one_hot_label3 = tf.one_hot(indices=tf.cast(y3, tf.int32), depth=CHAR_SET_LEN)

    # 计算loss
    loss0 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits0, labels=one_hot_label0))
    loss1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits1, labels=one_hot_label1))
    loss2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits2, labels=one_hot_label2))
    loss3 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits3, labels=one_hot_label3))
    
    # 计算总的loss
    total_loss = (loss0 + loss1 + loss2 + loss3) / 4.0
    optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(total_loss)
    
    # 计算准确率
    correct_prediction0 = tf.equal(tf.argmax(one_hot_label0, 1), tf.argmax(logits0, 1))
    accuracy0 = tf.reduce_mean(tf.cast(correct_prediction0, tf.float32))
    
    correct_prediction1 = tf.equal(tf.argmax(one_hot_label1, 1), tf.argmax(logits1, 1))
    accuracy1 = tf.reduce_mean(tf.cast(correct_prediction1, tf.float32))
    
    correct_prediction2 = tf.equal(tf.argmax(one_hot_label2, 1), tf.argmax(logits2, 1))
    accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))
    
    correct_prediction3 = tf.equal(tf.argmax(one_hot_label3, 1), tf.argmax(logits3, 1))
    accuracy3 = tf.reduce_mean(tf.cast(correct_prediction3, tf.float32))
    
    # 用于保存模型
    saver = tf.train.Saver()
    
    # 初始化
    sess.run(tf.global_variables_initializer())
    
    # 创建一个协调器，管理线程
    coord = tf.train.Coordinator()
    # 启动QueueRunner，此时文件名队列已经进队
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    
    for i in range(6001):
        # 获取一个批次的数据和标签
        b_image, b_label0, b_label1, b_label2, b_label3 = sess.run([image_batch, label_batch0, label_batch1, label_batch2, label_batch3])
        # 优化模型
        sess.run(optimizer, feed_dict={x: b_image, y0: b_label0, y1: b_label1, y2: b_label2, y3: b_label3})
        
        # 每迭代20次计算一次loss和准确率
        if i % 200 == 0:
            # 每迭代2000次降低一次学习率
            if i % 2000 == 0:
                sess.run(tf.assign(lr, lr/3))
            acc0, acc1, acc2, acc3, loss_ = sess.run([accuracy0, accuracy1, accuracy2, accuracy3, total_loss], feed_dict={x: b_image, y0: b_label0, y1: b_label1, y2: b_label2, y3: b_label3})
            
            learning_rate = sess.run(lr)
            
            print "Iter: %d  Loss: %.3f  Accuracy: %.2f, %.2f, %.2f, %.2f  Learning_rate: %.4f" % (i, loss_, acc0, acc1, acc2, acc3, learning_rate)
            
            if i % 6000 == 0:
                saver.save(sess, './captcha/model/crack_captcha.model', global_step=i)
    
    # 通知其他线程关闭
    coord.request_stop()
    # 其他所有线程关闭后，这一函数才能返回
    coord.join(threads)

In [ ]:
# 下面是我在GPU服务器的训练结果
# 代码整理于 https://github.com/zhulf0804/Verification-Code-Recognition

Iter: 0  Loss: 2696.982  Accuracy: 0.16, 0.20, 0.28, 0.24  Learning_rate: 0.0010
Iter: 200  Loss: 2.302  Accuracy: 0.20, 0.08, 0.04, 0.12  Learning_rate: 0.0010
Iter: 400  Loss: 2.312  Accuracy: 0.16, 0.12, 0.04, 0.00  Learning_rate: 0.0010
Iter: 600  Loss: 2.320  Accuracy: 0.08, 0.16, 0.04, 0.04  Learning_rate: 0.0010
Iter: 800  Loss: 2.213  Accuracy: 0.28, 0.24, 0.08, 0.20  Learning_rate: 0.0010
Iter: 1000  Loss: 1.817  Accuracy: 0.28, 0.20, 0.32, 0.52  Learning_rate: 0.0010
Iter: 1200  Loss: 1.456  Accuracy: 0.48, 0.28, 0.56, 0.40  Learning_rate: 0.0010
Iter: 1400  Loss: 1.037  Accuracy: 0.56, 0.52, 0.60, 0.76  Learning_rate: 0.0010
Iter: 1600  Loss: 0.947  Accuracy: 0.84, 0.56, 0.36, 0.88  Learning_rate: 0.0010
Iter: 1800  Loss: 0.507  Accuracy: 0.84, 0.76, 0.76, 0.88  Learning_rate: 0.0010
Iter: 2000  Loss: 0.601  Accuracy: 0.80, 0.76, 0.68, 0.72  Learning_rate: 0.0003
Iter: 2200  Loss: 0.469  Accuracy: 0.84, 0.88, 0.76, 0.92  Learning_rate: 0.0003
Iter: 2400  Loss: 0.427  Accuracy: 0.92, 0.80, 0.80, 0.96  Learning_rate: 0.0003
Iter: 2600  Loss: 0.425  Accuracy: 0.88, 0.76, 0.88, 0.96  Learning_rate: 0.0003
Iter: 2800  Loss: 0.176  Accuracy: 0.96, 0.96, 1.00, 0.88  Learning_rate: 0.0003
Iter: 3000  Loss: 0.202  Accuracy: 0.92, 0.96, 0.84, 0.96  Learning_rate: 0.0003
Iter: 3200  Loss: 0.243  Accuracy: 0.96, 0.96, 0.92, 0.80  Learning_rate: 0.0003
Iter: 3400  Loss: 0.172  Accuracy: 0.92, 0.96, 0.92, 0.96  Learning_rate: 0.0003
Iter: 3600  Loss: 0.342  Accuracy: 0.92, 0.92, 0.92, 0.92  Learning_rate: 0.0003
Iter: 3800  Loss: 0.150  Accuracy: 0.96, 0.96, 0.92, 0.96  Learning_rate: 0.0003
Iter: 4000  Loss: 0.177  Accuracy: 0.92, 0.88, 0.96, 0.96  Learning_rate: 0.0001
Iter: 4200  Loss: 0.125  Accuracy: 1.00, 0.92, 0.96, 0.92  Learning_rate: 0.0001
Iter: 4400  Loss: 0.114  Accuracy: 1.00, 0.92, 1.00, 1.00  Learning_rate: 0.0001
Iter: 4600  Loss: 0.128  Accuracy: 1.00, 1.00, 0.84, 1.00  Learning_rate: 0.0001
Iter: 4800  Loss: 0.100  Accuracy: 1.00, 0.92, 0.96, 0.96  Learning_rate: 0.0001
Iter: 5000  Loss: 0.075  Accuracy: 1.00, 0.96, 1.00, 0.96  Learning_rate: 0.0001
Iter: 5200  Loss: 0.061  Accuracy: 0.96, 1.00, 0.96, 1.00  Learning_rate: 0.0001
Iter: 5400  Loss: 0.098  Accuracy: 1.00, 0.96, 0.88, 1.00  Learning_rate: 0.0001
Iter: 5600  Loss: 0.145  Accuracy: 1.00, 0.96, 0.88, 0.96  Learning_rate: 0.0001
Iter: 5800  Loss: 0.090  Accuracy: 1.00, 1.00, 0.96, 0.96  Learning_rate: 0.0001
Iter: 6000  Loss: 0.079  Accuracy: 1.00, 1.00, 0.92, 0.96  Learning_rate: 0.0000
